<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [ ]:
!pip install ta
!pip install pandas
!pip install plotly
!pip install chart_studio

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=97b9a636ba57bb4884823295d24803d7e048f4f011dab62e830f0c83a3109fc4
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta
     |████████████████████████████████| 64 kB 2.6 MB/s 


Import Libs

In [ ]:
import pandas as pd
from ta.momentum import RSIIndicator

In [ ]:
def get_back_candles(df, index, back_count):
  loc = df.index.get_loc(index)
  return df.iloc[loc-1-back_count:loc-1]

## Load 15 min and 60 min stock data and calculate RSI

In [ ]:
file_15min = '/content/stock_data/ACC-HIST.csv'
file_60min = '/content/stock_data/ACC-HIST-60M.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 14:15:00+05:30,2233.45,2233.45,2224.50,2226.90,4832,32.784869
2021-12-17 14:30:00+05:30,2226.90,2228.00,2221.10,2223.25,6433,30.579297
2021-12-17 14:45:00+05:30,2223.25,2223.25,2210.50,2213.25,8311,25.514844
2021-12-17 15:00:00+05:30,2213.90,2222.90,2211.00,2211.00,23626,24.530431
2021-12-17 15:15:00+05:30,2212.00,2216.90,2207.95,2214.75,23281,29.418200


In [ ]:
df_60min = pd.read_csv(file_60min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_60min['Close']).rsi()
df_60min = df_60min.assign(rsi=rsi.values)
df_60min.tail()

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 11:15:00+05:30,2235.75,2253.70,2229.50,2236.00,47682,26.014586
2021-12-17 12:15:00+05:30,2236.00,2237.50,2223.30,2225.25,36768,23.305519
2021-12-17 13:15:00+05:30,2225.25,2240.00,2220.50,2233.45,58482,29.349308
2021-12-17 14:15:00+05:30,2233.45,2233.45,2210.50,2211.00,43202,23.815831
2021-12-17 15:15:00+05:30,2212.00,2216.90,2207.95,2214.75,23281,26.314907


## Set Strategy Parameters

In [ ]:
window_start = '2021-12-01 00:00:00'
window_end = '2021-12-18 00:00:00'
rsi_15min = 70
rsi_60min = 50
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [ ]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-03 09:15:00+05:30,2268.10,2285.50,2268.10,2281.10,57402,72.802384
2021-12-03 09:30:00+05:30,2282.60,2292.20,2276.55,2288.20,24645,75.689276
2021-12-03 09:45:00+05:30,2286.50,2298.00,2283.45,2291.75,34481,77.003634
2021-12-03 10:00:00+05:30,2291.90,2304.55,2286.95,2300.55,41519,79.904068
2021-12-03 10:45:00+05:30,2290.00,2300.00,2287.35,2297.30,11056,70.075666
2021-12-03 11:00:00+05:30,2297.30,2301.00,2296.30,2298.00,6002,70.356459
2021-12-08 10:15:00+05:30,2265.50,2279.00,2260.00,2277.45,26721,70.712706
2021-12-08 10:30:00+05:30,2277.70,2287.70,2275.00,2286.00,28245,73.883156
2021-12-08 10:45:00+05:30,2286.00,2293.00,2285.00,2292.00,21495,75.858218



### Step 4: Volume check

In [ ]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = get_back_candles(curr_window_df, index, back_candles)
  # print('Current candle:', index)
  # print('Back candles:', back_candles_df)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-12-03 09:15:00+05:30,2268.10,2285.5,2268.1,2281.10,57402.0,72.802384,19309.2
2021-12-08 15:15:00+05:30,2293.40,2299.9,2290.0,2299.90,20873.0,73.492074,10398.4
2021-12-09 15:00:00+05:30,2317.05,2324.0,2315.8,2323.05,60259.0,73.242854,6733.2


### Step 5: Hourly candle RSI check

In [ ]:
temp_df = pd.DataFrame(columns=df.columns)
hourly_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'
hourly_df.index.name = 'Date'
for index, row in df.iterrows():
  # Get the corresponding hourly candle for given 15min candle
  if(index in df_60min.index):
    # Candle already exists
    rsi = df_60min.loc[index]['rsi']
    hourly_df.loc[index] = df_60min.loc[index]
  else:
    # Update the index to the previous hourly candle index
    fromTime = index-pd.Timedelta(45,unit='min')
    temp_row = df_60min[df_60min.index >= fromTime].iloc[0]
    # print(temp_df)
    rsi = temp_row['rsi']
    hourly_df.loc[fromTime] = temp_row

  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    # print('Hourly candle RSI is greater than 50..')
    temp_df.loc[index] = row

df = temp_df
df,hourly_df

(                              Open    High     Low    Close   Volume        rsi
 Date                                                                           
 2021-12-03 09:15:00+05:30  2268.10  2285.5  2268.1  2281.10  57402.0  72.802384
 2021-12-08 15:15:00+05:30  2293.40  2299.9  2290.0  2299.90  20873.0  73.492074
 2021-12-09 15:00:00+05:30  2317.05  2324.0  2315.8  2323.05  60259.0  73.242854,
                              Open     High  ...    Volume        rsi
 Date                                        ...                     
 2021-12-03 09:15:00+05:30  2268.1  2304.55  ...  158047.0  56.930049
 2021-12-08 15:15:00+05:30  2293.4  2299.90  ...   20873.0  66.143761
 2021-12-09 14:15:00+05:30  2308.6  2324.00  ...   90923.0  71.832380
 
 [3 rows x 6 columns])

### Step 6: Stock movement check

In [ ]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candles = get_back_candles(df_15min, index, 5)
  min_low = prev_candles['Low'].min()
  daily_movement = row['High'] - min_low
  tanaji_pct = (daily_movement/min_low)*100
  print('daily_movement:', daily_movement,'tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

temp_df

daily_movement: 44.09999999999991 tanaji_pct: 1.967520299812613
daily_movement: 17.84999999999991 tanaji_pct: 0.7821914506693503
daily_movement: 26.90000000000009 tanaji_pct: 1.1710417482913278


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-03 09:15:00+05:30,2268.10,2285.5,2268.1,2281.10,57402.0,72.802384
2021-12-08 15:15:00+05:30,2293.40,2299.9,2290.0,2299.90,20873.0,73.492074
2021-12-09 15:00:00+05:30,2317.05,2324.0,2315.8,2323.05,60259.0,73.242854


## **Final Signal**

In [ ]:
for index, row in df.iterrows():
  prev_candle = get_back_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-12-03 09:15:00+05:30 , rsi: 72.80238434926991 Entry Price 2285.5 Stop Loss: 2256.4
For candle: 2021-12-08 15:15:00+05:30 , rsi: 73.49207375579942 Entry Price 2299.9 Stop Loss: 2284.0
For candle: 2021-12-09 15:00:00+05:30 , rsi: 73.24285432968023 Entry Price 2324.0 Stop Loss: 2305.0


In [ ]:
import plotly.graph_objects as go
import plotly.offline as py

# df.reset_index(inplace=True)
# print(df['Date'])
'''data=[go.Candlestick(x=df['Date'],
                     open=df['Open'],
                     high=df['High'],
                     low=df['Low'],
                     close=df['Close'])]

py.iplot(data)
'''

for index, row in df.iterrows():
  temp1 = get_back_candles(df_15min, index, 6)
  candles = pd.concat([temp1])
  candles.loc[index] = row

  candles['DateStr'] = candles.index.strftime('%Y-%m-%d %H:%M:%S')

  fig=go.Figure(data=[go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'])])
  fig.update_xaxes(type='category', rangeslider=dict(visible=False))

  py.iplot(fig)